<a href="https://colab.research.google.com/github/johanhoffman/DD2363-VT20/blob/leoenge/Lab-2/leoenge_lab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Lab 2: Matrix factorization**
**Leo Enge**

# **Abstract**

In this lab different matrix algorithmes were implemented and tested. They were often tested against the numpy library and all the tests succeded to several digits of precision.



#**About the code**

A short statement on who is the author of the file, and if the code is distributed under a certain license. 

In [0]:
"""This program is a template for lab reports in the course"""
"""DD2363 Methods in Scientific Computing, """
"""KTH Royal Institute of Technology, Stockholm, Sweden."""

# Copyright (C) 2019 Leo Enge (leoe@kth.se)

# This lab was done cooperating with Christoffer Ejemyr

# This file is part of the course DD2363 Methods in Scientific Computing
# KTH Royal Institute of Technology, Stockholm, Sweden
#
# This is free software: you can redistribute it and/or modify
# it under the terms of the GNU Lesser General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.

'KTH Royal Institute of Technology, Stockholm, Sweden.'

# **Set up environment**

To have access to the neccessary modules you have to run this cell. If you need additional modules, this is where you add them. 

In [0]:
# Load neccessary modules.
from google.colab import files

import time
import numpy as np
import scipy.sparse as sparse

from matplotlib import pyplot as plt
from matplotlib import tri
from matplotlib import axes
from mpl_toolkits.mplot3d import Axes3D

import unittest

# **Introduction**

In this lab different matrix algorithms are implemented, such as factorization, equation system solvning, least square solutions and blocked matrix multiplication.



# **Methods**

## Sparse matrix multiplication
Sparse matrix multiplication is a method to calculate the multiplication of a matrix and a vector when the matrix consists to large part of zeros. Numpy defines a class for sparse matrices which is overwritten in this lab. The method acts by saving the non-zero elemnents in the matrix and doing the operations only for those values.

In [0]:
class SparseMatrix(sparse.csr_matrix):
  def dot(self, vector):
    if not (type(vector) == np.ndarray and vector.ndim == 1 and vector.size == self.shape[1]):
      raise Exception("Vector must be a numpy-array of ndim 1 and compatible with the matrix")
    b = np.zeros(self.shape[0])
    for j in range(b.size):
      for i in range(self.indptr[j], self.indptr[j+1]):
        b[j] += self.data[i] * vector[self.indices[i]]
    return b
  
  def __str__(self):
    return str(self.todense())


### Testing the CSR multiplication
Even though the CSR multiplication is intended for sparse matrices, it should work for arbitrary matrices. Therefore the tests are made for 100 random matrices of random sizes.

In [0]:
class Test(unittest.TestCase):
  def test_common_errors(self):
    A = SparseMatrix([[1,0,0],[0,4,0],[0,1,0]])
    B = SparseMatrix([[0,0,1],[2,0,0],[0,1,0]])
    v1 = np.array([1,1,1,1])
    v2 = [1,2,1]
    with self.assertRaises(Exception):
      A.dot(B)
    with self.assertRaises(Exception):
      A.dot(v1)
    with self.assertRaises(Exception):
      A.dot(v2)
  
  def test_accuray(self):
    for _ in range(0,100):
      dim = np.random.randint(1,10)
      M = np.random.rand(np.random.randint(1,10), dim)
      M_csr = SparseMatrix(M)
      v = np.random.rand(dim)
      #Testing for which decimal value it fails to equal. Do not however care about more than 7 decimals.
      for i in range(3,7):
        np.testing.assert_almost_equal(M.dot(v), M_csr.dot(v), decimal=i)

if __name__ == '__main__':
  unittest.main(argv=['first-arg-is-ignored'], exit=False)

..
----------------------------------------------------------------------
Ran 2 tests in 0.101s

OK


## QR factorization
QR factorization done with the ordinary Gram-Schmidt method. It works by succesivley creating a orthonormal basis for the range of the original matrix, using Gram-Schmidt. These base vectors then make up the orthogonal matrix Q. 

This method was chosen because it is very intuitive and easy to understand.

In [0]:
def qr_factorization(matrix):
  if not (type(matrix) == np.ndarray and matrix.ndim == 2):
    raise Exception("The input matrix is bad...")
  
  n = matrix.shape[1]
  v = np.zeros(matrix.shape[0])
  v[:] = matrix[:,0]
  R = np.zeros((matrix.shape[1], matrix.shape[1]))
  Q = np.zeros(matrix.shape)

  for i in range(n):
    R[i,i] = np.linalg.norm(v)
    Q[:,i] = v/R[i,i]

    for j in range(i+1, n):
      R[i,j] = Q[:,i].dot(matrix[:,j])
    
    if i>n-2:
      break

    v[:] = matrix[:,i+1]
    for j in range(i+1):
      v[:] = v[:] - R[j,i+1]*Q[:,j]
    
  return Q,R

### Testing QR factorization

In [0]:
class Test(unittest.TestCase):
  
  def test_accuray(self):
    count = 0
    while count < 100:
      dim = np.random.randint(1,10)
      M = np.random.rand(dim, dim)
      if np.linalg.det(M) == 0:
        continue
      count += 1
      Q,R = qr_factorization(M)
      #Testing for which decimal value it fails to equal. Do not however care about more than 7 decimals.
      for i in range(3,7):
        np.testing.assert_almost_equal(Q.dot(R), M, decimal=i)
        np.testing.assert_almost_equal(np.linalg.norm(Q.transpose().dot(Q)-np.eye(dim), 'fro'),0, decimal=i)
        np.testing.assert_almost_equal(np.linalg.norm(Q.dot(R) - M,'fro'), 0, decimal=i)
        np.testing.assert_almost_equal(R, np.triu(R), decimal=i)

if __name__ == '__main__':
  unittest.main(argv=['first-arg-is-ignored'], exit=False)

.F
FAIL: test_singular (__main__.Test)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-88-0cdb7c53e006>", line 5, in test_singular
    qr_factorization(A)
AssertionError: Exception not raised

----------------------------------------------------------------------
Ran 2 tests in 0.170s

FAILED (failures=1)


## Equation system solver
Here an equation system solver of the system $Ax = b$ by first QR-factorize the matrix $A$. It is then easy to invert $Q$ since it is a orthogonal matrix, it will just be the transpose. So we get
\begin{equation}
R x = Q^T b
\end{equation}
where $R$ is upper triangular, so the system can be solved using backward substitution.

In [0]:
def backward_substitution(matrix, vector):
  if not (type(matrix) == np.ndarray and matrix.ndim == 2 and np.linalg.det(matrix) != 0):
    raise Exception("The input matrix is bad...")
  if not (type(vector) == np.ndarray and vector.ndim == 1):
    raise Exception("The input vector is bad...")
  if not(matrix.shape[0] == vector.size):
    raise Exception("The dimensions of the vector and the matrix are not compatible.")
  
  n = vector.size
  result = np.zeros(n)
  result[-1] = vector[-1]/matrix[-1,-1]
  for i in range(n-2, -1, -1):
    sum = 0
    for j in range(i+1, n):
      sum += matrix[i,j]*result[j]
    result[i] = (vector[i] - sum)/matrix[i,i]
  return result


In [0]:
def equation_system_solver(matrix, vector):
  if not (type(matrix) == np.ndarray and matrix.ndim == 2 and np.linalg.det(matrix) != 0):
    raise Exception("The input matrix is bad...")
  if not (type(vector) == np.ndarray and vector.ndim == 1):
    raise Exception("The input vector is bad...")
  if not(matrix.shape[0] == vector.size):
    raise Exception("The dimensions of the vector and the matrix are not compatible.")
  
  n = vector.size
  Q, R = qr_factorization(matrix)
  return backward_substitution(R, Q.transpose().dot(vector))

### Testing Equation System Solver
The equation solver is tested both by calculating the residual $\| Ax-b \|$ and also by calculating the "reverse" residual, which is given by $\| y-x\|$ where $y$ is an arbitrary vector and $x$ is the solution to the system $Ax = Ay$. 

In [0]:
class Test(unittest.TestCase):
  def test_singular(self):
    A = np.array([[1,0,1],[0,1,1],[1,0,1]])
    with self.assertRaises(Exception):
      qr_factorization(A)
  
  def test_first_residual(self):
    count = 0
    while count < 100:
      dim = np.random.randint(1,10)
      A = np.random.rand(dim, dim)
      if np.linalg.det(M) == 0:
        continue
      count += 1
      b = np.random.rand(dim)
      x = equation_system_solver(A, b)
      #Testing for which decimal value it fails to equal. Do not however care about more than 7 decimals.
      for i in range(3,7):
        np.testing.assert_almost_equal(A.dot(x), b, decimal=i)
  
  def test_reversed_residual(self):
    count = 0
    while count < 100:
      dim = np.random.randint(1,10)
      A = np.random.rand(dim, dim)
      if np.linalg.det(M) == 0:
        continue
      count += 1
      x = np.random.rand(dim)
      b = A.dot(x)
      y = equation_system_solver(A, b)
      #Testing for which decimal value it fails to equal. Do not however care about more than 7 decimals.
      for i in range(3,7):
        np.testing.assert_almost_equal(x, y, decimal=i)

if __name__ == '__main__':
  unittest.main(argv=['first-arg-is-ignored'], exit=False)

..F
FAIL: test_singular (__main__.Test)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-91-2beff27b9515>", line 5, in test_singular
    qr_factorization(A)
AssertionError: Exception not raised

----------------------------------------------------------------------
Ran 3 tests in 0.150s

FAILED (failures=1)


## Least square
The least square solution $y$ of $Ax = b$ is calculated using the Penrose-Moore inverse, which is given by
\begin{equation}
A^T A y =  A^T b
\end{equation}
this system is solved in the same manner as for the equation system solver above. The acual system solver above is not used, since it only accepts non-singular matrices.

In [0]:
def least_square(matrix, vector):
  if not (type(matrix) == np.ndarray and matrix.ndim == 2):
    raise Exception("The input matrix is bad...")
  if not (type(vector) == np.ndarray and vector.ndim == 1):
    raise Exception("The input vector is bad...")
  if not(matrix.shape[0] == vector.size):
    raise Exception("The dimensions of the vector and the matrix are not compatible.")
  Q,R = qr_factorization(matrix.transpose().dot(matrix))
  return backward_substitution(R, Q.transpose().dot(matrix.transpose().dot(vector)))

### Test Least Square
The least square is tested by checking the residual $\| Ay-b \|$ is smaller than $\|A(y+\Delta) - b\|$ for a lot of different $\Delta$. I.e. we check that $y$ is the solution that minimizes $\| Ay-b \|$.

In [0]:
class Test(unittest.TestCase):
  def test_accuracy(self):
    for _ in range(100):
      dim = np.random.randint(1,30)
      A = np.random.rand(dim, np.random.randint(1,dim+1))
      b = np.random.rand(dim)
      y = least_square(A, b)
      for _ in range(1000):
        delta = (np.random.rand(y.size) - 0.5)*2
        self.assertTrue(np.linalg.norm(A.dot(y)-b) <= np.linalg.norm(A.dot(y+delta)-b))


if __name__ == '__main__':
  unittest.main(argv=['first-arg-is-ignored'], exit=False)

.
----------------------------------------------------------------------
Ran 1 test in 2.392s

OK


## QR Eigenvalue Algorithm
An iterative algorithm for finding the eigenvalues of a square matrix is implemented much like algorithm 6.1 in the lecutre notes. 

In [0]:
def qr_eigenvalue(A, no_of_itterations):
  if not A.shape[0] == A.shape[1]:
    raise Exception("Matrix must be square.")
  A_result = np.array(A)
  U = np.eye(A.shape[0])
  for i in range(no_of_itterations):
      Q, R = qr_factorization(A_result)
      A_result = R.dot(Q)
      U = U.dot(Q)
  return A_result.diagonal(), U

In [0]:
class Test(unittest.TestCase):

  def test_determinant(self):
    for _ in range(100):
      dim = np.random.randint(1,10)
      A = np.random.rand(dim, dim)
      A = A.dot(A.transpose())
      B, U = qr_eigenvalue(A, 100)
      for i in range(3,7):
        for j in range(B.size):
          np.testing.assert_almost_equal(np.linalg.det(A - B[j]*np.eye(dim)), 0, decimal=i)


  def test_accuracy(self):
    for _ in range(100):
      dim = np.random.randint(1,10)
      A = np.random.rand(dim, dim)
      A = A.dot(A.transpose())
      B, U = qr_eigenvalue(A, 100)
      for i in range(3,7):
        for j in range(B.size):
          np.testing.assert_almost_equal(np.linalg.norm(A.dot(U[:,j]) - B[j]*U[:,j]), 0, decimal=i)

if __name__ == '__main__':
  unittest.main(argv=['first-arg-is-ignored'], exit=False)

FF
FAIL: test_accuracy (__main__.Test)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-95-1e37d3ff9d9d>", line 22, in test_accuracy
    np.testing.assert_almost_equal(np.linalg.norm(A.dot(U[:,j]) - B[j]*U[:,j]), 0, decimal=i)
  File "/usr/local/lib/python3.6/dist-packages/numpy/testing/_private/utils.py", line 616, in assert_almost_equal
    raise AssertionError(_build_err_msg())
AssertionError: 
Arrays are not almost equal to 5 decimals
 ACTUAL: 3.902884836664517e-05
 DESIRED: 0

FAIL: test_determinant (__main__.Test)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-95-1e37d3ff9d9d>", line 11, in test_determinant
    np.testing.assert_almost_equal(np.linalg.det(A - B[j]*np.eye(dim)), 0, decimal=i)
  File "/usr/local/lib/python3.6/dist-packages/numpy/testing/_private/utils.py", line 616, in assert_almost_equal
    raise Asserti

## Blocked Matrix Multiplication
The blocked matrix multiplication of $C = AB$ is done by multiplying matching blocks of $A$ and $B$ successivley, where both the matrices are divided in to a given set of blocks along both the rows and the columns.


In [0]:
def blocked_matrix_multiplication(A, B, n: int, m:int, p:int):
  if not (type(A) == np.ndarray and A.ndim == 2):
    raise Exception("The input matrix A is bad...")
  if not (type(B) == np.ndarray and B.ndim == 2):
    raise Exception("The input matrix B is bad...")
  if not (A.shape[1] == B.shape[0]):
    raise Exception("Matrix dimensions are not compatible for multiplication")
  if not (n>0 and m>0 and p>0 and n<=A.shape[0] and m <= B.shape[1] and p<=A.shape[1]):
    raise Exception("No. of blocks are too small or too large.")

  i_0, di = 0,0
  C = np.zeros((A.shape[0], B.shape[1]))

  for i in range(n):
    i_0 = i_0 + di
    di = int(np.ceil((A.shape[0]-di)/(n-i)))
    j_0,dj = 0,0
    for j in range(m):
      j_0 = j_0 + dj
      dj = int(np.ceil((B.shape[1]-dj)/(m-j)))
      k_0,dk = 0,0
      for k in range(p):
        k_0 = k_0 + dk
        dk = int(np.ceil(A.shape[1]-dk)/(p-k))
        C[i_0:i_0+di, j_0:j_0+dj] += A[i_0:i_0+di, k_0:k_0+dk].dot(B[k_0:k_0+dk, j_0:j_0+dj])
    
  return C



### Testing the blocked matrix multiplication

In [0]:
class Test(unittest.TestCase):
  def test_accuracy(self):
    for _ in range(100):
      dim_p = np.random.randint(1,30)
      dim_n = np.random.randint(1,30)
      dim_m = np.random.randint(1,30)
      A = np.random.rand(dim_n, dim_p)
      B = np.random.rand(dim_p, dim_m)
      n = np.random.randint(1,dim_n+1)
      m = np.random.randint(1,dim_m+1)
      p = np.random.randint(1,dim_p+1)
      x = blocked_matrix_multiplication(A,B,n,m,p)
      y = A.dot(B)
      for i in range(3,7):
        np.testing.assert_almost_equal(x, y, decimal=i)


if __name__ == '__main__':
  unittest.main(argv=['first-arg-is-ignored'], exit=False)

.
----------------------------------------------------------------------
Ran 1 test in 0.480s

OK


# **Results**

All the tests succeded to at least the seventh decimal, except the QR eigenvalue algorithm, which succeded the fourth or fifth decimal, depending on test and on run.

# **Discussion**

Overall the algorithms worked as expected, without any big problems. 

